# 국방조달

In [1]:
# don't need to load extension everytime. 
%load_ext autoreload 

# This needs to be executed everytime.  
%autoreload 2

In [2]:

from org.d2b import *
driver = d2b_create_driver()


[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 103.0.1264
[WDM] - Get LATEST edgedriver version for 103.0.1264 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/103.0.1264.62/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Jaeyoung\.wdm\drivers\edgedriver\win64\103.0.1264.62]


# Login

- https://www.d2b.go.kr/index.do



In [3]:

%autoreload 2

from auto.auto import *
import account

import org.d2b.d2b 

def login(driver: WebDriver, token='BIO-SEAL') -> bool:
    driver.get("https://www.d2b.go.kr/index.do")

    if d2b._is_login(driver):
        return True

    # login button
    auto_click(driver, By.ID, "_mLogin")

    # 지문인식 로그인 버튼
    # XXX: 너무 빨리 click이 되면 문제가 발생한다.
    #       인증 프로그램 실행 준비가 안되었습니다. 설치가 안된 경우 제품을 설치 후 진행해 주시기 바랍니다
    # TODO: 적절한 수준 찾기
    fp_login_btn = auto_find_element(driver, By.ID, "_fingerLoginBtn")
    time.sleep(3)
    auto_click(driver, fp_login_btn)

    # alert 창 확인 버튼
    alert = auto_find_alert(driver, timeout=3)
    alert.accept()

    # 지문 토큰
    auto_click(driver, By.ID, "NX_MEDIA_BIOHSM")

    # 지문 토큰 종류 선택 - BIO
    selection = auto_find_element(
        driver, By.XPATH, '//*[@id="nx-cert-select"]/div[3]/div[1]/div[3]')
    res = auto_wait_until(lambda: selection.is_displayed())
    if not res:
        print("failed")
    else:
        # XXX: 로딩된 이후 디폴트 값이 선택되기 까지 기다린다. 바로 선택하게 되면 디폴트 값으로 돌아가 버린다.
        time.sleep(1)

    # 사용자로 부터 전달 받은 token을 선택
    auto_click(driver, By.XPATH,
               f'//div[@id="cert-select-area3"]/table/tbody/tr/td[contains(text(), "{token}")]')

    # 확인버튼
    auto_click(driver, By.XPATH,
               '//*[@id="pki-extra-media-box-contents3"]/div[3]/button')

    #####################
    # 사용자의 지문 입력 #
    #####################

    # Pin 번호 입력 - display될 때까지 기다린다.
    pin_input = auto_find_element(driver, By.ID, 'nx_cert_pin')
    auto_wait_until(lambda: pin_input.is_displayed(), timeout=120)
    auto_type(pin_input, "00000000")
    auto_click(driver, By.XPATH,
               '//*[@id="pki-extra-media-box-contents3"]/div[2]/button')

    # 확인 버튼
    auto_click(driver, By.XPATH, '//*[@id="nx-cert-select"]/div[4]/button[1]')

    # popup 입찰서 작성안내 (optional)
    button = auto_find_element(driver, By.ID, '_closeBtn1')
    if button:
        auto_click(driver, button)


login(driver, token = 'BIO-SEAL')



UnexpectedAlertPresentException: Alert Text: [ERROR CODE :357695488]
세션 연결에 실패하였습니다. 미디어 장치[보안토큰 & 바이오 토큰] 연결 상태를 확인하세요. PC에서 해당 미디어를 인식하지 못합니다. 드라이버를 재설치 하거나 제조업체에 문의하세요.
Message: unexpected alert open: {Alert text : [ERROR CODE :357695488]
세션 연결에 실패하였습니다. 미디어 장치[보안토큰 & 바이오 토큰] 연결 상태를 확인하세요. PC에서 해당 미디어를 인식하지 못합니다. 드라이버를 재설치 하거나 제조업체에 문의하세요.}
  (Session info: MicrosoftEdge=103.0.1264.62)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::unpack [0x00007FF797A66312+24642]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00007FF7979B9A62+334098]
	Ordinal0 [0x00007FF797557915+620821]
	Ordinal0 [0x00007FF7975C2E39+1060409]
	Ordinal0 [0x00007FF7975AECB3+978099]
	Ordinal0 [0x00007FF797585F00+810752]
	Ordinal0 [0x00007FF797587508+816392]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF79781D578+135480]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF797804C77+34871]
	Microsoft::Applications::Events::EventProperty::Ev

# Pre Registration

In [75]:





def d2b_register(driver: WebDriver, number, user, cert_pw):
    # move to the page to register
    driver.get('https://www.d2b.go.kr/index.do')
    # type number and click search button
    auto_type(driver, By.ID, "numb_divs", number)
    auto_click(driver, By.ID, 'btn_search')

    # 검색된 결과 중 첫번째 element를 선택한다.
    notice = auto_find_element(
        driver, By.XPATH, '//*[@id="SBHE_DATAGRID_WHOLE_TBODY_datagrid1"]/tr[2]/td[8]/div/span/a')
    log().info(
        f"Found notice from number. notice={notice.text}, number={number}")
    # auto_click(driver, notice) -> 안됨...
    notice.click()

    # 입찰참가신청서 작성
    auto_click(driver, By.ID, 'btn_join')

    # 신청서 작성후 popup이 생성 된다면.. 이미 신청이 된 상태이다.
    alert = auto_find_alert(driver, timeout=3)
    if alert:
        if alert.text.find("입찰참가등록이 미완료된 건") >= 0:
            alert.accept()
        else:
            print(f"Already registered. text={alert.text}")
            alert.accept()
            return True

    # 서약서 작성
    auto_click(driver, By.ID, 'c_box1')
    auto_click(driver, By.ID, 'c_box2')
    auto_click(driver, By.ID, 'subcont_dir_pay_yn1')
    # XXX: Need to wait the above result?
    # time.sleep(3)
    auto_click(driver, By.ID, 'btn_confirm')

    # 보증금납부 방법
    sel = Select(auto_find_element(driver, By.ID, 'grnt_mthd'))
    sel.select_by_visible_text('보증금면제')

    # ID: D2b - reg - 보증금 동의문
    # 보증금납부에 대한 서약서 확인
    auto_click(driver, By.ID, 'c_box2')
    auto_click(driver, By.ID, 'c_box3')
    auto_click(driver, By.XPATH,
               '//div[5]/div[2]/div[2]/div/div/div[3]/button[1]')

    # 약관 동의 체크
    auto_click(driver, By.ID, 'bidAttention_check')

    # 신청 버튼
    auto_click(driver, By.ID, 'btn_wrt')
    alert = auto_find_alert(driver)
    log().info(f"Accept alert! text={alert.text}")
    alert.accept()

    # 인증서 로그인
    d2b_cert_login(driver, user, cert_pw)

    # 팝업 확인
    alert = auto_find_alert(driver)
    log().info(f"Accept alert! text={alert.text}")
    alert.accept()

    # ID: D2B - 사후심사대상 입찰안내    
    # Not Always
    # ACTION: "닫기" 버튼이 있고, 보인다면 클릭.
    close_btn = auto_find_element(driver, By.XPATH, '//*[@id="layer"]/div[2]/div/div/div[2]/button[3]')
    if close_btn and close_btn.is_displayed():
        close_btn.click()

    return True




## Participate

In [76]:

import random
# def participate(driver:WebDriver, number):


# 2022UMM048321246
# 판단번호: 21246, 21905 (심사중..)
# 판단번호: 24 (사후심사)

# ID: D2B - 서약서동의(2)
def _agree_oath_2(driver:WebDriver):

    # - 조세포탈 없음을 확약하는 서약서
    if not auto_click(driver, By.ID, 'c_box1'):
        return False

    # - 청렴 계약 이행 서약서    
    if not auto_click(driver, By.ID, 'c_box2'):
        return False

    # - 행정정보 공동이용 사전동의서
    if not auto_click(driver, By.ID, 'c_box3'):
        return False
    
    # 확인 버튼
    if not auto_click(driver, By.ID, 'btn_oath_confirm'):
        return False
    
    return True

def _go_bid_detail_page(driver:WebDriver, number):
    # move to the page to register
    driver.get('https://www.d2b.go.kr/index.do')
    # type number and click search button
    auto_type(driver, By.ID, "numb_divs", number)
    auto_click(driver, By.ID, 'btn_search')

    # 검색된 결과 중 첫번째 element를 선택한다.
    notice = auto_find_element(
        driver, By.XPATH, '//*[@id="SBHE_DATAGRID_WHOLE_TBODY_datagrid1"]/tr[2]/td[8]/div/span/a')
    if not notice:
        log().error(f"Failed to search bid item. number={number}")
        return False

    log().info(
        f"Found notice from number. notice={notice.text}, number={number}")
    # auto_click(driver, notice) -> 안됨...
    notice.click()
    return True



# ID: D2B - 견적서 작성(w/o 참가신청)
# Precondition: 견적서작성 페이지여야 한다. 
def _write_estimate(driver:WebDriver, cost) -> bool:
    # 1. 견적금액 작성
    if not auto_type(driver, By.ID,"input_amount", cost):
        log().error("Failed to find 견적금액 input")
        return False

    # 2. 복수예비가격 선택: 2개
    check_boxes = auto_find_all_elements(driver, By.XPATH, '//input[@name="check_multi_price"]')
    check_boxes = random.choices(check_boxes, k=2)
    for box in check_boxes:
        print(box)
        box.click()

    time.sleep(3)

    # 3. 제출 버튼 click
    if not auto_click(driver, By.ID, "btn_submit"):
        log().error("Failed to find 제출 button")
        return False

    # 4. 견적서 제출 확인 popup
    alert = auto_find_alert(driver)
    if not alert:
        log().error("Failed to find 제출 확인 메세지")
        return False

    log().debug(f"Accept. text={alert.text}")
    alert.accept()

    return True


def _need_registration(driver:WebDriver, number) -> bool:
    # 1. 상세 페이지로 이동
    _go_bid_detail_page(driver, number)

    # 2. "견적서작성" vs. "입찰참가신청" 버튼
    # 견적서작성 버튼이 있으면 입찰참가가 필요 없다. 
    btn = auto_find_element(driver, By.XPATH, '//button[@id="btn_estimate_write"]', timeout=5)
    return True if not btn else False

# ID: D2B - 입찰참가 불필요건에 대한 견적서 작성
# Condition: 입찰건 detail page에서 "견적서작성" 버튼이 있어야 한다.
def _participate_without_registration(driver:WebDriver, number, cost) -> bool:

    # 1. 아이템을 검색한다. 
    _go_bid_detail_page(driver, number)

    # 2. 견적서 작성 버튼을 찾는다. 
    estimate_write_btn = auto_find_element(driver, By.XPATH, '//button[@id="btn_estimate_write"]', timeout=5)
    if not estimate_write_btn:
        log().debug("Failed to find 견적서작성 button.")
        return False
    estimate_write_btn.click()

    # 3. 서약서 작성
    if not _agree_oath_2(driver):
        log().error("Failed to write oath(2).")
        return False

    # 4. 견적서 작성
    if not _write_estimate(driver, cost):
        log().error("Failed to write estimate.")
        return False
    
    # 5. 견적서 제출 확인 - 견적서를 성공적으로 제출하였습니다. 
    alert = auto_find_alert(driver)    
    alert.accept()

    return True


# ID: D2b - 입찰(참가신청) - 참가가능 검증
# number form example: "MDR0033-1", "UMM0424-1", "UMM0483-1"
def _can_participate(driver, number):
    x = auto_find_element(driver, By.XPATH, '//a[text()="물품"]/../div')
    driver.execute_script("arguments[0].setAttribute('style',arguments[1])", x, "display: block;")

    auto_click(driver, By.XPATH, '//a[text()="입찰"]')
    auto_click(driver, By.XPATH, '//h5/a[text()="참가신청서 조회"]')

    notices = auto_find_all_elements(driver, By.XPATH, '//tbody[@id="SBHE_DATAGRID_WHOLE_TBODY_datagrid1"]/tr')
    notices = notices[1:]
    
    for notice in notices:
        if not notice or notice.find_element(By.XPATH, './/td[2]/div/span').text.find(number) < 0:
            continue
        
        status = notice.find_element(By.XPATH, './/td[7]/div/span').text
        if status.find("투찰가능") < 0:
            log().error(f"Can not participate. number={number}, status={status}")
            return False
        
        return True

    log().error(f"Can not find notice. number={number}")
    return False


def _write_bid(driver, cost):
    # 1. cost 입력
    auto_type(driver, By.ID, "bid_amnt_1", cost)
    
    # 2. 추첨 checkbox 선택(2개)
    boxes = auto_find_all_elements(driver, By.XPATH, '//td/input[@type="checkbox"]')
    if len(boxes) != 15:
        log().error(f"Can not find all 추첨확인 checkbox.len={len(boxes)}")
        return False
    boxes = random.sample(boxes, 2)
    for box in boxes:
        box.click()

    if not auto_click(driver, By.ID, "c_box"):
        log().error(f"Can not find 동의 checkbox.")
        return False
    
    return True
    
def _sumbit_bid(driver):
    if not auto_click(driver, By.ID, "btn_bid_submit"):
        log().error(f"Can not find 제출 버튼.")
        return False

    alert = wait_alert(driver)
    log().info(f"accpet. alert={alert.text}")
    alert.accept()

    alert = wait_alert(driver)
    log().info(f"accpet. alert={alert.text}")
    alert.accept()

    return True
    
    
    

# Precondition:
#  - 입찰서제출 및 조회(경쟁입찰) 페이지 내에서 실행 되어야 한다. 
# WARNING: number format is different
def choose_bid_in_list(driver:WebDriver, number):
    # 공고번호 선택    
    notices = auto_find_all_elements(driver, By.XPATH, '//tbody[@id="SBHE_DATAGRID_WHOLE_TBODY_datagrid1"]/tr')
    notice = next(filter(lambda x: x.find_element(By.XPATH, ".//td[1]/div").text.find(number) >= 0, notices[1:]), None)
    if not notice:
        print(f"Can not find the bid. number={number}, cost={cost}")
        return False
    notice.click()

    # validate - Selected? 
    if notice.find_element(By.XPATH, './/td[1]').get_attribute("sbgrid_select") != "true":
        log().error(f"Can not find the bid. number={number}, cost={cost}")
        return False

    # validate - can participate? 
    status = notice.find_element(By.XPATH, './/td[6]/div/span').text 
    if status != "미제출":
        log().error(f"Fail to verify its status. number= {number}, status={status}")
        return False

    return True

# ID: D2b - 입찰(참가신청) - 입찰서 작성 페이지 이동
def _go_to_bid_write_page(driver, number):
    # ID: D2b - 입찰(참가신청) - 입찰서 작성 페이지이동
    # 페이지 이동
    if not auto_click(driver, By.XPATH, '//h5/a[text()="입찰서제출 및 조회(경쟁입찰)"]'):
        log().error(f"Failed to find 입찰서제출 및 조회(경쟁입찰) button")
        return False

    # 페이지내 관련 공고 선택
    if not choose_bid_in_list(driver, number[4:11]):
        log().error(f"Failed to go to the bid page")
        return False
    
    # 입찰서 작성 버튼 클릭
    if not auto_click(driver, By.ID, 'btn_bid_regi'):
        log().error(f"Failed to find 입찰서 작성 button")
        return False

    return True


def _participate_with_registration(driver, number, cost):
    # validate it's ready
    # WARNING: number format is different
    # ID: D2b - 입찰(참가신청) - 참가가능 검증
    if not _can_participate(driver, number[4:11]):
        log().error(f"Failed to validate the bid. number={number}, cost={cost}")
        return False
    
    # ID: D2b - 입찰(참가신청) - 입찰서 작성 페이지 이동
    if not _go_to_bid_write_page(driver, number) :
        log().error(f". number={number}, cost={cost}")
        return False

    # ID: D2b - 입찰(참가신청) - 입찰서 작성

    # 입찰서 작성
    if not _write_bid(driver, cost):
        log().error(f"Failed to write a bid form. number={number}, cost={cost}")
        return False

    # 입찰서 제출
    if not _sumbit_bid(driver):
        log().error(f"Failed to submit a bid. number={number}, cost={cost}")
        return False

    return True



def participate(driver, number, cost):
    if not _need_registration(driver, number):
        return _participate_without_registration(driver, number, cost)
    else:
        return _participate_with_registration(driver, number, cost)





In [77]:




# user = account.account_get("d2b", "user")
# cert = account.account_get("d2b", "cert")
# # number = '2022UMM042421817-01'
# number = '2022UMM048321246-01'
# d2b_register(driver, number, user, cert)


# 2022UMM048321246

number = '2022UMM042421817-01'
cost = "55199804"
participate(driver, number, cost)


Failed to submit a bid. number=2022UMM042421817-01, cost=55199804


False

# Login Test

In [1]:

from account import account_get
from org.d2b import D2B

d2b_id = account_get("d2b", "id")
d2b_pw = account_get("d2b", "pw")
d2b_user = account_get("d2b", "user")
d2b_cert = account_get("d2b", "cert")

d2b = D2B(d2b_id, d2b_pw, d2b_user, d2b_cert, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 104.0.1293
[WDM] - Get LATEST edgedriver version for 104.0.1293 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/104.0.1293.54/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Jaeyoung\.wdm\drivers\edgedriver\win64\104.0.1293.54]


Error element not found
